# Đọc file và kiểm tra

In [27]:
import os 


sound_folder = 'Sound/'
folder_list = os.listdir(sound_folder)

for folder in folder_list:
    print("Folder: " + folder)
    for file in os.listdir(sound_folder + folder + '/'):

        print(file)

Folder: Bye
Bye.wav
Bye2.wav
Bye3.wav
Folder: Hello


# Biến đổi đầu vào và gắn nhãn

In [24]:
import librosa


#create label_list and mfcc_list
mfcc_list = []
label_list = []


# each folder is label because each label have many file sound
for folder in folder_list:
    for file in os.listdir(sound_folder + folder + '/'):
        y, sr = librosa.load(os.path.join(sound_folder, folder, file), sr = None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12)

        label_list.append(folder)
        mfcc_list.append(mfcc)

        print(folder)
        print(sr)
        print(y)
        print(mfcc)
        print()


        

Bye
44100
[ 0.          0.          0.         ... -0.01409912 -0.00601196
  0.00161743]
[[-6.57796143e+02 -6.55372375e+02 -5.10442444e+02 ... -3.62398865e+02
  -3.57399841e+02 -3.82482605e+02]
 [ 0.00000000e+00  3.40845704e+00  1.12403076e+02 ...  1.47081009e+02
   1.45921356e+02  1.28070129e+02]
 [ 0.00000000e+00  3.35104012e+00  1.99178200e+01 ... -7.23037491e+01
  -7.53158417e+01 -7.34255447e+01]
 ...
 [ 0.00000000e+00  2.03142929e+00  7.58084297e+00 ...  4.68315840e+00
   6.41852188e+00  1.23945799e+01]
 [ 0.00000000e+00  1.74869871e+00  1.82782245e+00 ... -1.46227765e+00
   2.67781317e-01  3.68009567e+00]
 [ 0.00000000e+00  1.45452952e+00  6.76789761e-01 ... -1.94259224e+01
  -1.56595421e+01 -9.52735329e+00]]

Bye
44100
[0.         0.         0.         ... 0.00976562 0.00881958 0.00695801]
[[-617.8179    -617.8179    -509.7673    ... -454.3055    -454.16464
  -458.85153  ]
 [   0.           0.          58.80503   ...  118.09656    120.66124
   121.236786 ]
 [   0.           0.  

# Sử lý dữ liệu đầu vào

In [29]:
import numpy as np

# Số đặc trưng mong muốn
desired_num_features = 200
processed_mfcc_list = []
# Cắt hoặc zero-padding các khung MFCC để có cùng số đặc trưng
def process_mfcc(mfcc):
    if mfcc.shape[1] > desired_num_features:
        mfcc = mfcc[:, :desired_num_features]
    elif mfcc.shape[1] < desired_num_features:
        num_missing_features = desired_num_features - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, num_missing_features)), mode='constant')
    return mfcc


for mfcc in mfcc_list:
    processed_mfcc = process_mfcc(mfcc)
    processed_mfcc_list.append(processed_mfcc)


    
for mfcc in processed_mfcc_list:
    print(mfcc.shape)

(12, 200)
(12, 200)
(12, 200)


# Huấn luyện bằng mô hình RNN


In [35]:
import tensorflow as tf


label_numeric = [i for i, _ in enumerate(label_list)]
label_arr = np.asarray(label_numeric)
mfcc_arr = np.asarray(processed_mfcc_list)



model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32),  # Sử dụng LSTM với 64 units
    tf.keras.layers.Dense(units = 32, activation='softmax')  
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(mfcc_arr, label_arr, epochs=10, batch_size=32, validation_split=0.01)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 4.0081 - accuracy: 0.0000e+00 - val_loss: 3.5748 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 35ms/step - loss: 3.3698 - accuracy: 0.0000e+00 - val_loss: 3.6992 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 34ms/step - loss: 3.0200 - accuracy: 0.0000e+00 - val_loss: 3.5810 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 34ms/step - loss: 2.8419 - accuracy: 0.0000e+00 - val_loss: 3.4766 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 41ms/step - loss: 2.6433 - accuracy: 0.5000 - val_loss: 3.4060 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 34ms/step - loss: 2.5579 - accuracy: 0.5000 - val_loss: 3.3233 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 34ms/step - loss: 2.4752 - accuracy: 0.5000 - val_loss: 3.2551 - val_accura